# Stable Diffusion in KerasCV (Colab)



## Overview

KerasCV 라이브러리에 구현된 [stability.ai](https://stability.ai/) 의 text-to-image 모델,
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) 을 사용해 봅니다.

Stable Diffusion 은 이미지 생성 분야의 지각변동을 일으킨 **오픈소스** text-to-image 생성모델입니다.  
오픈소스 프로젝트이기 때문에, 여러 프로젝트에 구현체들이 존재합니다.
KerasCV 의 구현체는 [XLA compilation](https://www.tensorflow.org/xla) 과
[mixed precision](https://www.tensorflow.org/guide/mixed_precision) 기능을 이용하여 속도를 올렸습니다.  
해당 기능들은 Tensorflow 플랫폼에서 지원합니다.  

직접 Stable Diffusion 모델을 받아서 사용해보도록 하겠습니다.  

In [1]:
# windows 에서는 module import error 발생. resource module 을 windows 에서 호환되도록 준비가 필요합니다.

!pip install tensorflow keras_cv --upgrade --quiet

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\jhpark\\anaconda3\\Lib\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import time
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt

KeyboardInterrupt: 

## Introduction

`keras_cv.models.StableDiffusion()` method 를 이용하여 미리 준비된 Stable Diffusion 모델을 가져옵니다.


In [ ]:
model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)

프롬프트에 아래와 같은 문구를 넣어봅시다.

In [ ]:
images = model.text_to_image("photograph of an astronaut (Samsung Electronics) riding a horse", batch_size=3)


def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")


plot_images(images)

보다 복잡한 prompt 도 넣어봅시다.  
Keyword 들을 잘 나열하면 원하는 결과물을 얻기 쉽습니다.  

In [ ]:
images = model.text_to_image(
    "cute magical flying dog, fantasy art, "
    "golden color, high quality, highly detailed, elegant, sharp focus, "
    "concept art, character concepts, digital painting, mystery, adventure",
    batch_size=3,
)
plot_images(images)

## 동작은 어떻게 되는 것인가요?



Stalbe Diffusion 은 아래 3가지 파트로 요약할 수 있습니다.

- Text Encoder, 프롬프트를 인코딩하여 vector 로 만듧니다.
- Diffusion Model, 디노이징을 해서 64x64 이미지 vector (Latent Image Patch) 를 만듦니다.
- Decoder, 64x64 Image Patch를 고화질 512x512 이미지로 만듦니다.

디노이징은 step을 반복하는데, step을 많이 할 수록 이미지 품질이 좋아집니다.  
기본 셋팅은 50 스텝으로 되어있습니다.


![The Stable Diffusion architecture](https://i.imgur.com/2uC8rYJ.png)

실제 keras 의 구현은 아래 코드에서 볼 수 있고, 매우 간단합니다.


- [text_encoder.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/text_encoder.py): 87 LOC
- [diffusion_model.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/diffusion_model.py): 181 LOC
- [decoder.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/decoder.py): 86 LOC
- [stable_diffusion.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/stable_diffusion.py): 106 LOC



(원 저자)  
**Authors:** [fchollet](https://twitter.com/fchollet), [lukewood](https://twitter.com/luke_wood_ml), [divamgupta](https://github.com/divamgupta)<br>
**Date created:** 2022/09/25<br>
**Last modified:** 2022/09/25<br>